In [ ]:
!pip install optuna
!pip install boruta

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import ADASYN
import optuna
import joblib

# Read data
data = pd.read_excel('TrainClass.xlsx')
real_data = pd.read_excel('TestClass.xlsx')

# Split data
Y, X = data['FRACASO'], data.drop(['FRACASO', 'CODIGO_EMPRESA'], axis=1)

# Apply ADASYN
adasyn = ADASYN(random_state=0)
X_resampled, Y_resampled = adasyn.fit_resample(X, Y)

X_train, X_test, Y_train, Y_test = train_test_split(X_resampled, Y_resampled, test_size=0.2, random_state=0)
X = real_data.drop(['CODIGO_EMPRESA'], axis=1)

# Define the objective function for cross-validated AUC
def objective(trial):
    # Define hyperparameters to be optimized
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 1, 10)

    # Create the classifier with suggested hyperparameters
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=0)

    # Boruta feature selection
    boruta_selector = BorutaPy(clf, n_estimators='auto', random_state=0)
    boruta_selector.fit(X_train.values, Y_train.values)

    # Get selected features
    selected_features = X_train.columns[boruta_selector.support_]

    # Define cross-validation strategy (StratifiedKFold for classification)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

    # Perform cross-validation and get mean AUC
    auc_scores = cross_val_score(clf, X_train[selected_features], Y_train, cv=cv, scoring='roc_auc')
    mean_auc = auc_scores.mean()

    return mean_auc

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params
print(f'Best Hyperparameters: {best_params}')

# Train the final model with the best hyperparameters
best_clf = RandomForestClassifier(**best_params, random_state=0)

# Boruta feature selection on the entire dataset
boruta_selector = BorutaPy(best_clf, n_estimators='auto', random_state=0)
boruta_selector.fit(X_train.values, Y_train.values)

# Get selected features
selected_features = X_train.columns[boruta_selector.support_]

X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]
X_selected = X[selected_features]

best_clf.fit(X_train_selected, Y_train)

# Predictions on training set
y_pred_train = best_clf.predict(X_train_selected)

# Calculate AUC for training set
auc_train = roc_auc_score(Y_train, y_pred_train)
print(f'AUC for Training Set: {auc_train:.4f}')

# Predictions on testing set
y_pred_test = best_clf.predict(X_test_selected)

# Calculate AUC for testing set
auc_test = roc_auc_score(Y_test, y_pred_test)
print(f'AUC for Testing Set: {auc_test:.4f}')

# Save the trained model to a file
joblib.dump(best_clf, 'random_forest_model_boruta.joblib')

# Load model and generate probability CSV
loaded_model = joblib.load('random_forest_model_boruta.joblib')
prob = loaded_model.predict_proba(X_selected)[:, 1]
prob_df = pd.DataFrame(prob, columns=['Probability'])
prob_df.index = prob_df.index + 1
prob_df.index.name = 'Id'
prob_df.to_csv('intento_boruta.csv', index=True)


[I 2023-12-05 16:40:10,596] A new study created in memory with name: no-name-1fcebbb4-8810-4a70-bb63-7502befe2a77
[I 2023-12-05 16:40:46,846] Trial 0 finished with value: 0.9102722072543947 and parameters: {'n_estimators': 199, 'max_depth': 4}. Best is trial 0 with value: 0.9102722072543947.
[I 2023-12-05 16:41:14,392] Trial 1 finished with value: 0.9102722072543947 and parameters: {'n_estimators': 163, 'max_depth': 4}. Best is trial 0 with value: 0.9102722072543947.
[I 2023-12-05 16:41:39,294] Trial 2 finished with value: 0.9370487020615462 and parameters: {'n_estimators': 61, 'max_depth': 5}. Best is trial 2 with value: 0.9370487020615462.
[I 2023-12-05 16:42:00,453] Trial 3 finished with value: 0.9914492708074487 and parameters: {'n_estimators': 82, 'max_depth': 10}. Best is trial 3 with value: 0.9914492708074487.
[I 2023-12-05 16:43:07,183] Trial 4 finished with value: 0.8634719923317714 and parameters: {'n_estimators': 53, 'max_depth': 2}. Best is trial 3 with value: 0.99144927080

Best Hyperparameters: {'n_estimators': 82, 'max_depth': 10}
AUC for Training Set: 0.9942
AUC for Testing Set: 0.9593
